In [31]:
# import tensorflow library
import tensorflow as tf
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
# one_hot key implies lables in onehot encoding
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [32]:
learning_rate = 0.001
training_iters = 5000
batch_size = 128
display_step = 10

In [33]:
# Network Parameters
n_input = 784 # 28*28
n_classes = 10 # (0-9 digits)
dropout = 0.75 # What use?: to make network neuron more independent....

In [34]:
# define placeholders
x = tf.placeholder(tf.float32, shape=(None, n_input))
y = tf.placeholder(tf.float32, shape=(None, n_classes))

In [35]:
# miscill... functions
def define_variable(shape, name): 
    return tf.Variable(tf.truncated_normal(shape, name = name))

def conv2d(x, W, b):
    x = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [36]:
weights = {
    'wc1': define_variable([5, 5, 1, 32], 'wc1'), # 5x5 conv, 1 input, 40 outputs 
    'wc2': define_variable([5, 5, 32, 64], 'wc2'), # 5x5 conv, 40 inputs, 80 outputs
    'wc3': define_variable([5, 5, 64, 128], 'wc3'), # 5x5 conv, 80 inputs, 500 outputs
    'wc4': define_variable([3, 3, 128, 516], 'wc4'), # 2x2 conv, 500 inputs, 1024 outputs
    'wc5': define_variable([3, 3, 516, 1024], 'wc5'), # 2x2 conv, 500 inputs, 1024 outputs
    'out': define_variable([1024, n_classes], 'out') # 1024 inputs, 10 outputs (class prediction)
}

In [37]:
biases = {
    'bc1': define_variable([32], 'bc1'),
    'bc2': define_variable([64], 'bc2'),
    'bc3': define_variable([128], 'bc3'),
    'bc4': define_variable([516], 'bc4'),
    'bc5': define_variable([1024], 'bc5'),
    'out': define_variable([n_classes], 'out')
}

In [38]:
def main_net(x, weights, biases, dropout = dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    pool1 = maxpool2d(conv1) # feature map dimension : 14x14

    conv2 = conv2d(pool1, weights['wc2'], biases['bc2'])
    pool2 = maxpool2d(conv2) # feature map dimension : 7x7
    print pool2
    
    conv3 = conv2d(pool2, weights['wc3'], biases['bc3'])
    pool3 = maxpool2d(conv3) # feature map dimension : 4x4
    print pool3 
    
    conv4 = conv2d(pool3, weights['wc4'], biases['bc4'])
    pool4 = maxpool2d(conv4) # feature map dimension : 2x2 
    print pool4
    
    conv5 = conv2d(pool4, weights['wc5'], biases['bc5'])
    pool5 = maxpool2d(conv5)
    print pool5
    
    unroll = tf.reshape(pool5, [-1, weights['out'].get_shape().as_list()[0]])
    print unroll
    
    out = tf.add(tf.matmul(unroll, weights['out']), biases['out'])
    return out

In [39]:
pred = main_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

Tensor("MaxPool_16:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("MaxPool_17:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("MaxPool_18:0", shape=(?, 2, 2, 516), dtype=float32)
Tensor("MaxPool_19:0", shape=(?, 1, 1, 1024), dtype=float32)
Tensor("Reshape_16:0", shape=(?, 1024), dtype=float32)


In [40]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})
            print("loss= {:.6f}".format(loss) + ", Accuracy= {:.5f}".format(acc))
        step += 1
    writer = tf.summary.FileWriter("/abcd", graph=tf.get_default_graph())
    print("Optimization Finished!")
    
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256]}))

loss= 14808062.000000, Accuracy= 0.33594
loss= 4725499.000000, Accuracy= 0.49219
loss= 2436205.500000, Accuracy= 0.67188
Optimization Finished!
('Testing Accuracy:', 0.66796875)
